In [13]:
import xarray as xr 

ds = xr.open_dataset('soil-water-forecasting/modeling-pipeline/data/02_intermediate/preprocessed_data.nc')

In [14]:
ds

<xarray.Dataset> Size: 504MB
Dimensions:     (valid_time: 84, latitude: 181, longitude: 360)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 672B 2016-01-01 ... 2022-12-01
  * latitude    (latitude) float64 1kB -90.0 -89.0 -88.0 ... 88.0 89.0 90.0
  * longitude   (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/23)
    cl          (valid_time, latitude, longitude) float32 22MB ...
    cvh         (valid_time, latitude, longitude) float32 22MB ...
    tvh         (valid_time, latitude, longitude) float32 22MB ...
    asn         (valid_time, latitude, longitude) float32 22MB ...
    rsn         (valid_time, latitude, longitude) float32 22MB ...
    sst         (valid_time, latitude, longitude) float32 22MB ...
    ...          ...
    ssr         (valid_time, latitude, longitude) float32 22MB ...
    e           (valid_time, latitude, longitude) float32 22MB ...
    ro          (valid_time, latitude, longitude) float32 22MB ...
    tp          (valid_time, latitude, longitude) float32 22MB ...
    pev         (valid_time, latitude, longitude) float32 22MB ...
    swvl1       (valid_time, latitude, longitude) float32 22MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

Temporal Features

Lets add Seasonal Cycles

In [15]:
# Extract the season from the time coordinate
ds['season'] = ds['valid_time'].dt.season

In [16]:
ds

<xarray.Dataset> Size: 504MB
Dimensions:     (valid_time: 84, latitude: 181, longitude: 360)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 672B 2016-01-01 ... 2022-12-01
  * latitude    (latitude) float64 1kB -90.0 -89.0 -88.0 ... 88.0 89.0 90.0
  * longitude   (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/24)
    cl          (valid_time, latitude, longitude) float32 22MB ...
    cvh         (valid_time, latitude, longitude) float32 22MB ...
    tvh         (valid_time, latitude, longitude) float32 22MB ...
    asn         (valid_time, latitude, longitude) float32 22MB ...
    rsn         (valid_time, latitude, longitude) float32 22MB ...
    sst         (valid_time, latitude, longitude) float32 22MB ...
    ...          ...
    e           (valid_time, latitude, longitude) float32 22MB ...
    ro          (valid_time, latitude, longitude) float32 22MB ...
    tp          (valid_time, latitude, longitude) float32 22MB ...
    pev         (valid_time, latitude, longitude) float32 22MB ...
    swvl1       (valid_time, latitude, longitude) float32 22MB ...
    season      (valid_time) <U3 1kB 'DJF' 'DJF' 'MAM' ... 'SON' 'SON' 'DJF'
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

In [17]:
# Define a dictionary to map season strings to numerical values
season_mapping = {'DJF': 1, 'MAM': 2, 'JJA': 3, 'SON': 4}

# Convert the DataArray to a pandas Series, map the season names, and convert back to a DataArray
ds['season'] = ds['season'].to_series().map(season_mapping).to_xarray()

In [18]:
ds

<xarray.Dataset> Size: 504MB
Dimensions:     (valid_time: 84, latitude: 181, longitude: 360)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 672B 2016-01-01 ... 2022-12-01
  * latitude    (latitude) float64 1kB -90.0 -89.0 -88.0 ... 88.0 89.0 90.0
  * longitude   (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/24)
    cl          (valid_time, latitude, longitude) float32 22MB ...
    cvh         (valid_time, latitude, longitude) float32 22MB ...
    tvh         (valid_time, latitude, longitude) float32 22MB ...
    asn         (valid_time, latitude, longitude) float32 22MB ...
    rsn         (valid_time, latitude, longitude) float32 22MB ...
    sst         (valid_time, latitude, longitude) float32 22MB ...
    ...          ...
    e           (valid_time, latitude, longitude) float32 22MB ...
    ro          (valid_time, latitude, longitude) float32 22MB ...
    tp          (valid_time, latitude, longitude) float32 22MB ...
    pev         (valid_time, latitude, longitude) float32 22MB ...
    swvl1       (valid_time, latitude, longitude) float32 22MB ...
    season      (valid_time) int64 672B 1 1 2 2 2 3 3 3 4 ... 2 2 3 3 3 4 4 4 1
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

In [19]:
# Broadcast 'season' to match dimensions (date, latitude, longitude)
season_broadcasted = ds['season'].expand_dims({'latitude': ds.latitude, 'longitude': ds.longitude})

# Add this expanded 'season' variable back to the dataset
ds['season'] = season_broadcasted

# Transpose 'season' to match the desired dimensions (date, latitude, longitude)
ds['season'] = ds['season'].transpose('valid_time', 'latitude', 'longitude')

In [20]:
ds

<xarray.Dataset> Size: 547MB
Dimensions:     (valid_time: 84, latitude: 181, longitude: 360)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 672B 2016-01-01 ... 2022-12-01
  * latitude    (latitude) float64 1kB -90.0 -89.0 -88.0 ... 88.0 89.0 90.0
  * longitude   (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/24)
    cl          (valid_time, latitude, longitude) float32 22MB ...
    cvh         (valid_time, latitude, longitude) float32 22MB ...
    tvh         (valid_time, latitude, longitude) float32 22MB ...
    asn         (valid_time, latitude, longitude) float32 22MB ...
    rsn         (valid_time, latitude, longitude) float32 22MB ...
    sst         (valid_time, latitude, longitude) float32 22MB ...
    ...          ...
    e           (valid_time, latitude, longitude) float32 22MB ...
    ro          (valid_time, latitude, longitude) float32 22MB ...
    tp          (valid_time, latitude, longitude) float32 22MB ...
    pev         (valid_time, latitude, longitude) float32 22MB ...
    swvl1       (valid_time, latitude, longitude) float32 22MB ...
    season      (valid_time, latitude, longitude) int64 44MB 1 1 1 1 ... 1 1 1 1
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

In [21]:
# Convert season to int8
ds['season'] = ds['season'].astype('int32')

In [22]:
ds

<xarray.Dataset> Size: 525MB
Dimensions:     (valid_time: 84, latitude: 181, longitude: 360)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 672B 2016-01-01 ... 2022-12-01
  * latitude    (latitude) float64 1kB -90.0 -89.0 -88.0 ... 88.0 89.0 90.0
  * longitude   (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/24)
    cl          (valid_time, latitude, longitude) float32 22MB ...
    cvh         (valid_time, latitude, longitude) float32 22MB ...
    tvh         (valid_time, latitude, longitude) float32 22MB ...
    asn         (valid_time, latitude, longitude) float32 22MB ...
    rsn         (valid_time, latitude, longitude) float32 22MB ...
    sst         (valid_time, latitude, longitude) float32 22MB ...
    ...          ...
    e           (valid_time, latitude, longitude) float32 22MB ...
    ro          (valid_time, latitude, longitude) float32 22MB ...
    tp          (valid_time, latitude, longitude) float32 22MB ...
    pev         (valid_time, latitude, longitude) float32 22MB ...
    swvl1       (valid_time, latitude, longitude) float32 22MB ...
    season      (valid_time, latitude, longitude) int32 22MB 1 1 1 1 ... 1 1 1 1
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

Soil Moisture Dynamics

Evapotranspiration Balance: Calculate a moisture balance feature as tp - e (precipitation minus evaporation). This balance can serve as an indicator of moisture availability and stress, especially over rolling windows.

Wind Impact: Combine u10 and v10 (wind components) to create a wind speed variable to assess the drying effect of wind on soil moisture.

Determine Wind Direction: Calculate the wind direction using: Wind speed and direction influence evapotranspiration rates, affecting soil moisture levels.

Temperature Anomalies: Identify deviations from long-term average temperatures to capture unusual thermal conditions impacting soil moisture.

In [23]:
ds.to_netcdf('soil-water-forecasting/modeling-pipeline/data/04_feature/features.nc')